<a href="https://colab.research.google.com/github/NitzanEz/Final-Project/blob/main/classificationCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloning the project repository

In [ ]:
!git clone https://github.com/NitzanEz/Final-Project.git


Cloning into 'Final-Project'...
remote: Enumerating objects: 112825, done.
remote: Counting objects: 100% (434/434), done.
remote: Compressing objects: 100% (429/429), done.
remote: Total 112825 (delta 34), reused 5 (delta 5), pack-reused 112391 (from 9)
Receiving objects: 100% (112825/112825), 2.39 GiB | 23.69 MiB/s, done.
Resolving deltas: 100% (1481/1481), done.
Updating files: 100% (61832/61832), done.


Import necessary libraries and define preprocessing and convolutional operations


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D, Conv2D, AveragePooling2D
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Activation
from tensorflow.keras.layers import BatchNormalization, concatenate
from tensorflow.keras import regularizers, initializers, Model
from tensorflow.keras.utils import get_file
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Normalization


#########################################################################################
# Implements the Inception Network v4 (http://arxiv.org/pdf/1602.07261v1.pdf) in Keras. #
#########################################################################################

WEIGHTS_PATH = 'https://github.com/kentsommer/keras-inceptionV4/releases/download/2.1/inception-v4_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/kentsommer/keras-inceptionV4/releases/download/2.1/inception-v4_weights_tf_dim_ordering_tf_kernels_notop.h5'

def preprocess_input(x):

    x = np.divide(x, 255.0)
    x = np.subtract(x, 0.5)
    x = np.multiply(x, 2.0)
    return x

def conv2d_bn(x, nb_filter, num_row, num_col, padding='same', strides=(1, 1), use_bias=False):
    """
    Utility function to apply conv + BN.
    """
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1
    x = Conv2D(nb_filter, (num_row, num_col),
               strides=strides,
               padding=padding,
               use_bias=use_bias,
               kernel_regularizer=regularizers.l2(0.00004),
               kernel_initializer=initializers.VarianceScaling(scale=2.0, mode='fan_in', distribution='normal', seed=None))(x)
    x = BatchNormalization(axis=channel_axis, momentum=0.9997, scale=False)(x)
    x = Activation('relu')(x)
    return x

def block_inception_a(input):
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    branch_0 = conv2d_bn(input, 96, 1, 1)

    branch_1 = conv2d_bn(input, 64, 1, 1)
    branch_1 = conv2d_bn(branch_1, 96, 3, 3)

    branch_2 = conv2d_bn(input, 64, 1, 1)
    branch_2 = conv2d_bn(branch_2, 96, 3, 3)
    branch_2 = conv2d_bn(branch_2, 96, 3, 3)

    branch_3 = AveragePooling2D((3,3), strides=(1,1), padding='same')(input)
    branch_3 = conv2d_bn(branch_3, 96, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x

def block_reduction_a(input):
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    branch_0 = conv2d_bn(input, 384, 3, 3, strides=(2,2), padding='valid')

    branch_1 = conv2d_bn(input, 192, 1, 1)
    branch_1 = conv2d_bn(branch_1, 224, 3, 3)
    branch_1 = conv2d_bn(branch_1, 256, 3, 3, strides=(2,2), padding='valid')

    branch_2 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(input)

    x = concatenate([branch_0, branch_1, branch_2], axis=channel_axis)
    return x

def block_inception_b(input):
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    branch_0 = conv2d_bn(input, 384, 1, 1)

    branch_1 = conv2d_bn(input, 192, 1, 1)
    branch_1 = conv2d_bn(branch_1, 224, 1, 7)
    branch_1 = conv2d_bn(branch_1, 256, 7, 1)

    branch_2 = conv2d_bn(input, 192, 1, 1)
    branch_2 = conv2d_bn(branch_2, 192, 7, 1)
    branch_2 = conv2d_bn(branch_2, 224, 1, 7)
    branch_2 = conv2d_bn(branch_2, 224, 7, 1)
    branch_2 = conv2d_bn(branch_2, 256, 1, 7)

    branch_3 = AveragePooling2D((3,3), strides=(1,1), padding='same')(input)
    branch_3 = conv2d_bn(branch_3, 128, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x

def block_reduction_b(input):
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    branch_0 = conv2d_bn(input, 192, 1, 1)
    branch_0 = conv2d_bn(branch_0, 192, 3, 3, strides=(2, 2), padding='valid')

    branch_1 = conv2d_bn(input, 256, 1, 1)
    branch_1 = conv2d_bn(branch_1, 256, 1, 7)
    branch_1 = conv2d_bn(branch_1, 320, 7, 1)
    branch_1 = conv2d_bn(branch_1, 320, 3, 3, strides=(2,2), padding='valid')

    branch_2 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(input)

    x = concatenate([branch_0, branch_1, branch_2], axis=channel_axis)
    return x

def block_inception_c(input):
    channel_axis = -1 if tf.keras.backend.image_data_format() == 'channels_last' else 1

    branch_0 = conv2d_bn(input, 256, 1, 1)

    branch_1 = conv2d_bn(input, 384, 1, 1)
    branch_10 = conv2d_bn(branch_1, 256, 1, 3)
    branch_11 = conv2d_bn(branch_1, 256, 3, 1)
    branch_1 = concatenate([branch_10, branch_11], axis=channel_axis)

    branch_2 = conv2d_bn(input, 384, 1, 1)
    branch_2 = conv2d_bn(branch_2, 448, 3, 1)
    branch_2 = conv2d_bn(branch_2, 512, 1, 3)
    branch_20 = conv2d_bn(branch_2, 256, 1, 3)
    branch_21 = conv2d_bn(branch_2, 256, 3, 1)
    branch_2 = concatenate([branch_20, branch_21], axis=channel_axis)

    branch_3 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(input)
    branch_3 = conv2d_bn(branch_3, 256, 1, 1)

    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x

def inception_v4_base(input):
    if tf.keras.backend.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    norm_layer = Normalization(axis=-1)

    # Apply normalization to the input tensor
    x = norm_layer(input)

    # Proceed with the model architecture
    x = conv2d_bn(x, 32, 3, 3, strides=(2,2), padding='valid')

    x = conv2d_bn(x, 32, 3, 3, strides=(2, 2), padding='valid')
    net = conv2d_bn(input, 32, 3, 3, strides=(2,2), padding='valid')
    net = conv2d_bn(net, 32, 3, 3, padding='valid')
    net = conv2d_bn(net, 64, 3, 3)

    branch_0 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(net)

    branch_1 = conv2d_bn(net, 96, 3, 3, strides=(2,2), padding='valid')

    net = concatenate([branch_0, branch_1], axis=channel_axis)

    branch_0 = conv2d_bn(net, 64, 1, 1)
    branch_0 = conv2d_bn(branch_0, 96, 3, 3, padding='valid')

    branch_1 = conv2d_bn(net, 64, 1, 1)
    branch_1 = conv2d_bn(branch_1, 64, 1, 7)
    branch_1 = conv2d_bn(branch_1, 64, 7, 1)
    branch_1 = conv2d_bn(branch_1, 96, 3, 3, padding='valid')

    net = concatenate([branch_0, branch_1], axis=channel_axis)

    branch_0 = conv2d_bn(net, 192, 3, 3, strides=(2,2), padding='valid')
    branch_1 = MaxPooling2D((3,3), strides=(2,2), padding='valid')(net)

    net = concatenate([branch_0, branch_1], axis=channel_axis)

    # 4 x Inception-A blocks
    for i in range(4):
        net = block_inception_a(net)

    # Reduction-A block
    net = block_reduction_a(net)

    # 7 x Inception-B blocks
    for i in range(7):
        net = block_inception_b(net)

    # Reduction-B block
    net = block_reduction_b(net)

    # 3 x Inception-C blocks
    for i in range(3):
        net = block_inception_c(net)

    return net

def inception_v4(num_classes, dropout_keep_prob, weights, include_top):
    inputs = Input((299, 299, 3))
    x = inception_v4_base(inputs)

    if include_top:
        x = AveragePooling2D((8, 8), padding='valid')(x)
        x = Dropout(dropout_keep_prob)(x)
        x = Flatten()(x)
        x = Dense(units=num_classes, activation='softmax')(x)

    model = Model(inputs, x, name='inception_v4')

    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('inception-v4_weights_tf_dim_ordering_tf_kernels.h5', WEIGHTS_PATH, cache_subdir='models')
        else:
            weights_path = get_file('inception-v4_weights_tf_dim_ordering_tf_kernels_notop.h5', WEIGHTS_PATH_NO_TOP, cache_subdir='models')
        model.load_weights(weights_path, by_name=True)
    return model

def create_and_compile_model(num_classes=1001, dropout_prob=0.3, weights='imagenet', include_top=True, learning_rate=0.0001):
    model = inception_v4(num_classes, dropout_prob, weights, include_top)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Usage
model = create_and_compile_model(num_classes=1001, dropout_prob=0.3, weights='imagenet', include_top=True, learning_rate=0.0001)

In [1]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# Input layer definition
inputs = Input(shape=(299, 299, 3))
base_model_output = inception_v4_base(inputs)
x = GlobalAveragePooling2D()(base_model_output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# Model setup
model = Model(inputs, predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Directory paths for train, validation, and test datasets
train_dir = '/content/Final-Project/Data/train'
validation_dir = '/content/Final-Project/Data/validation'
test_dir = '/content/Final-Project/Data/test'

# Image data generator for training with data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Image data generator for validation and testing (no data augmentation)
test_val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data generators for train, validation, and test sets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_val_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_val_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

# Model training
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Save the trained model
model.save('final_model.keras')

# Optionally load and use the model
final_model = load_model('final_model.keras')

# Evaluate the model on the test set
test_loss, test_accuracy = final_model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy}, Test Loss: {test_loss}')


NameError: name 'inception_v4_base' is not defined

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.models import load_model
from PIL import Image

# Load your Inception v4 model
model = load_model('/content/best_model.keras')  # Correct path to your model

def preprocess_input(x):
    """Preprocess the input image for Inception v4."""
    x = np.divide(x, 255.0)  # Normalize to [0, 1]
    x = np.subtract(x, 0.5)  # Center to [-0.5, 0.5]
    x = np.multiply(x, 2.0)  # Scale to [-1, 1]
    return x

def load_and_pad_image(file_path, target_size=(299, 299)):
    """Load an image and pad it to the target size."""
    img = keras_image.load_img(file_path)
    img = img.resize(target_size, Image.Resampling.LANCZOS)  # Use LANCZOS for high-quality downsampling
    img_array = keras_image.img_to_array(img)
    img_array = preprocess_input(img_array)  # Custom preprocessing for Inception v4
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Path to your image
image_path = '/content/Final-Project/Data/test/ASD/28872/slice_136.jpg'  # Correct path to your image

# Load and pad the image
prepared_image = load_and_pad_image(image_path)

# Predict using the model
predictions = model.predict(prepared_image)
predicted_class = np.argmax(predictions, axis=1)  # Assuming your model outputs logits for each class

# Map the class indices back to class labels if necessary
class_labels = {0: 'Non-ASD', 1: 'ASD'}
predicted_label = class_labels[predicted_class[0]]

print(f"The image is classified as: {predicted_label}")


ValueError: File not found: filepath=/content/best_model.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
from PIL import Image
import io

# Try to open an image file directly with PIL
try:
    img = Image.open('path_to_problematic_image.jpg')
    img.show()  # This will display the image if it can be opened
except IOError:
    print("Cannot open image")


In [ ]:
!git add classificationCode.ipynb  # Change 'Notebook.ipynb' to your file name
!git commit -m "Add Jupyter notebook"
!git push
